In [ ]:
import pandas as pd
import os
import io
###
# Load Explore Data 
# Definir o diretório de trabalho
work_dir = "/home/ubuntu/tech_challenge"
os.makedirs(work_dir, exist_ok=True)

# Caminho para o dataset fornecido pelo utilizador
dataset_path = "Tech Challenge/insurance (1).csv"

# Caminho para guardar o resumo da exploração
summary_file_path = os.path.join(work_dir, "data_exploration_summary.txt")

try:
    # Verificar se o ficheiro existe
    if not os.path.exists(dataset_path):
        raise FileNotFoundError(f"O ficheiro {dataset_path} não foi encontrado.")

    # Carregar o dataset
    print(f"A carregar o dataset de {dataset_path}...")
    df = pd.read_csv(dataset_path)

    # Mostrar as primeiras linhas e informações básicas
    print("\nPrimeiras 5 linhas do dataset:")
    print(df.head())

    print("\nInformações do dataset:")
    # Capturar a saída de df.info() para o ficheiro
    buffer_info = io.StringIO()
    df.info(buf=buffer_info)
    info_str = buffer_info.getvalue()
    print(info_str)

    print("\nEstatísticas descritivas:")
    desc_stats = df.describe()
    print(desc_stats)

    # Guardar as informações e estatísticas num ficheiro
    with open(summary_file_path, "w") as f:
        f.write("Primeiras 5 linhas do dataset:\n")
        f.write(df.head().to_string())
        f.write("\n\nInformações do dataset:\n")
        f.write(info_str)
        f.write("\n\nEstatísticas descritivas:\n")
        f.write(desc_stats.to_string())

    print(f"\nResumo da exploração guardado em {summary_file_path}")

    # Guardar o dataframe limpo para a próxima etapa (neste caso, apenas carregado)
    # Nenhuma limpeza foi feita ainda, mas guardamos para manter o fluxo
    df.to_csv(os.path.join(work_dir, "01_loaded_data.csv"), index=False)
    print(f"Dataset carregado guardado em {os.path.join(work_dir, '01_loaded_data.csv')}")

except FileNotFoundError as fnf_error:
    print(f"Erro: {fnf_error}")
except Exception as e:
    print(f"Ocorreu um erro inesperado: {e}")




In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Visualize Pre-Process 
# Definir o diretório de trabalho e de gráficos
work_dir = "/home/ubuntu/tech_challenge"
plots_dir = os.path.join(work_dir, "plots")
os.makedirs(plots_dir, exist_ok=True)

# Caminho para o dataset carregado
loaded_data_path = os.path.join(work_dir, "01_loaded_data.csv")
# Caminho para guardar os dados pré-processados
preprocessed_data_path = os.path.join(work_dir, "02_preprocessed_data.csv")

try:
    # Carregar o dataset
    print(f"A carregar o dataset de {loaded_data_path}...")
    df = pd.read_csv(loaded_data_path)

    # --- Visualização ---
    print("A gerar visualizações...")

    # Histograma das variáveis numéricas
    numerical_cols = ['age', 'bmi', 'children', 'charges']
    df[numerical_cols].hist(bins=30, figsize=(12, 8))
    plt.suptitle('Histograma das Variáveis Numéricas')
    plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Ajustar para o título
    hist_path = os.path.join(plots_dir, "numerical_histograms.png")
    plt.savefig(hist_path)
    plt.close()
    print(f"Histogramas guardados em {hist_path}")

    # Countplot das variáveis categóricas - CORREÇÃO APLICADA
    categorical_cols = ['sex', 'smoker', 'region', 'children'] # Incluir children como categórica para visualização
    fig, axes = plt.subplots(1, len(categorical_cols), figsize=(20, 5))
    for i, col in enumerate(categorical_cols):
        # Correção: Usar hue=col e legend=False para manter o efeito visual da palette
        sns.countplot(x=col, data=df, ax=axes[i], hue=col, palette='viridis', legend=False)
        axes[i].set_title(f'Distribuição de {col}')
    plt.suptitle('Distribuição das Variáveis Categóricas')
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    count_path = os.path.join(plots_dir, "categorical_countplots.png")
    plt.savefig(count_path)
    plt.close()
    print(f"Countplots guardados em {count_path}")

    # Boxplot: Custo vs. Categóricas
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    sns.boxplot(x='sex', y='charges', data=df, ax=axes[0])
    axes[0].set_title('Custo vs. Sexo')
    sns.boxplot(x='smoker', y='charges', data=df, ax=axes[1])
    axes[1].set_title('Custo vs. Fumante')
    sns.boxplot(x='region', y='charges', data=df, ax=axes[2])
    axes[2].set_title('Custo vs. Região')
    plt.suptitle('Boxplot: Custo vs. Variáveis Categóricas')
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    box_path = os.path.join(plots_dir, "charges_vs_categorical_boxplots.png")
    plt.savefig(box_path)
    plt.close()
    print(f"Boxplots Custo vs Categóricas guardados em {box_path}")

    # Scatterplot: Custo vs. Numéricas (age, bmi)
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    sns.scatterplot(x='age', y='charges', data=df, hue='smoker', ax=axes[0])
    axes[0].set_title('Custo vs. Idade (por Fumante)')
    sns.scatterplot(x='bmi', y='charges', data=df, hue='smoker', ax=axes[1])
    axes[1].set_title('Custo vs. IMC (por Fumante)')
    plt.suptitle('Scatterplot: Custo vs. Variáveis Numéricas')
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    scatter_path = os.path.join(plots_dir, "charges_vs_numerical_scatterplots.png")
    plt.savefig(scatter_path)
    plt.close()
    print(f"Scatterplots Custo vs Numéricas guardados em {scatter_path}")

    # Matriz de Correlação (apenas numéricas)
    # Primeiro, converter categóricas para numéricas temporariamente para correlação
    df_corr = df.copy()
    df_corr['sex'] = df_corr['sex'].map({'female': 0, 'male': 1})
    df_corr['smoker'] = df_corr['smoker'].map({'no': 0, 'yes': 1})
    # Para região, podemos usar one-hot encoding ou ignorar na correlação direta
    # Vamos calcular a correlação apenas com as outras
    corr_matrix = df_corr[['age', 'sex', 'bmi', 'children', 'smoker', 'charges']].corr()
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
    plt.title('Matriz de Correlação')
    corr_path = os.path.join(plots_dir, "correlation_matrix.png")
    plt.savefig(corr_path)
    plt.close()
    print(f"Matriz de correlação guardada em {corr_path}")

    # --- Pré-processamento ---
    print("\nA iniciar pré-processamento...")

    # Verificar valores ausentes (já feito na exploração, mas confirmar)
    print("\nValores ausentes por coluna:")
    print(df.isnull().sum())
    # Nenhuma ação necessária se não houver valores ausentes

    # Converter variáveis categóricas em numéricas usando One-Hot Encoding
    print("\nA converter variáveis categóricas (sex, smoker, region) para formato numérico...")
    df_processed = pd.get_dummies(df, columns=['sex', 'smoker', 'region'], drop_first=True, dtype=int)
    # drop_first=True para evitar multicolinearidade

    print("\nDataset após conversão de categóricas (primeiras 5 linhas):")
    print(df_processed.head())

    # Guardar o dataset pré-processado
    df_processed.to_csv(preprocessed_data_path, index=False)
    print(f"\nDataset pré-processado guardado em {preprocessed_data_path}")

except FileNotFoundError as fnf_error:
    print(f"Erro: {fnf_error}")
except Exception as e:
    print(f"Ocorreu um erro inesperado durante a visualização/pré-processamento: {e}")

A carregar o dataset de /home/ubuntu/tech_challenge\01_loaded_data.csv...
A gerar visualizações...
Histogramas guardados em /home/ubuntu/tech_challenge\plots\numerical_histograms.png
Countplots guardados em /home/ubuntu/tech_challenge\plots\categorical_countplots.png
Boxplots Custo vs Categóricas guardados em /home/ubuntu/tech_challenge\plots\charges_vs_categorical_boxplots.png
Scatterplots Custo vs Numéricas guardados em /home/ubuntu/tech_challenge\plots\charges_vs_numerical_scatterplots.png
Matriz de correlação guardada em /home/ubuntu/tech_challenge\plots\correlation_matrix.png

A iniciar pré-processamento...

Valores ausentes por coluna:
age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

A converter variáveis categóricas (sex, smoker, region) para formato numérico...

Dataset após conversão de categóricas (primeiras 5 linhas):
   age     bmi  children      charges  sex_male  smoker_yes  region_northwest  \
0   19  27.900 

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import os
import io

# Model Train Evaluate 
# Definir diretórios
work_dir = "/home/ubuntu/tech_challenge"
plots_dir = os.path.join(work_dir, "plots")
results_dir = os.path.join(work_dir, "results")
os.makedirs(results_dir, exist_ok=True)

# Caminhos dos ficheiros
preprocessed_data_path = os.path.join(work_dir, "02_preprocessed_data.csv")
model_summary_path = os.path.join(results_dir, "model_summary.txt")
evaluation_metrics_path = os.path.join(results_dir, "evaluation_metrics.txt")
predictions_plot_path = os.path.join(plots_dir, "predictions_vs_actual.png")
residuals_plot_path = os.path.join(plots_dir, "residuals_plot.png")

try:
    # Carregar dados pré-processados
    print(f"A carregar dados pré-processados de {preprocessed_data_path}...")
    df_processed = pd.read_csv(preprocessed_data_path)

    # --- Modelagem ---
    print("\nA iniciar modelagem...")

    # Definir variáveis independentes (X) e dependente (y)
    X = df_processed.drop("charges", axis=1)
    y = df_processed["charges"]

    # Dividir dados em treino e teste (80% treino, 20% teste)
    print("A dividir dados em conjuntos de treino e teste...")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"Tamanho do conjunto de treino: {X_train.shape[0]} amostras")
    print(f"Tamanho do conjunto de teste: {X_test.shape[0]} amostras")

    # --- Treinamento (Regressão Linear) ---
    print("\nA treinar o modelo de Regressão Linear...")
    model = LinearRegression()
    model.fit(X_train, y_train)
    print("Modelo treinado com sucesso.")

    # --- Avaliação ---
    print("\nA avaliar o modelo no conjunto de teste...")
    y_pred = model.predict(X_test)

    # Calcular métricas de avaliação
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    print(f"Mean Absolute Error (MAE): {mae:.2f}")
    print(f"Mean Squared Error (MSE): {mse:.2f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
    print(f"R-squared (R²): {r2:.4f}")

    # Guardar métricas de avaliação
    with open(evaluation_metrics_path, "w") as f:
        f.write("Métricas de Avaliação do Modelo (Conjunto de Teste):\n")
        f.write(f"Mean Absolute Error (MAE): {mae:.2f}\n")
        f.write(f"Mean Squared Error (MSE): {mse:.2f}\n")
        f.write(f"Root Mean Squared Error (RMSE): {rmse:.2f}\n")
        f.write(f"R-squared (R²): {r2:.4f}\n")
    print(f"Métricas de avaliação guardadas em {evaluation_metrics_path}")

    # --- Validação Estatística (usando statsmodels para p-values e intervalos de confiança) ---
    print("\nA realizar validação estatística com statsmodels...")
    # Adicionar uma constante (intercepto) ao modelo
    X_train_sm = sm.add_constant(X_train)
    # Criar e treinar o modelo OLS (Ordinary Least Squares)
    model_sm = sm.OLS(y_train, X_train_sm).fit()

    # Obter o sumário do modelo
    model_summary = model_sm.summary()
    print("\nSumário do Modelo (statsmodels OLS):")
    print(model_summary)

    # Guardar o sumário do modelo
    with open(model_summary_path, "w") as f:
        f.write("Sumário do Modelo (statsmodels OLS - Treino):\n")
        f.write(model_summary.as_text())
    print(f"Sumário do modelo guardado em {model_summary_path}")

    # --- Apresentação de Resultados Visuais ---
    print("\nA gerar gráficos de resultados...")

    # Gráfico: Previsões vs. Valores Reais
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, y_pred, alpha=0.6)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--r', linewidth=2)
    plt.xlabel("Valores Reais (Charges)")
    plt.ylabel("Previsões (Charges)")
    plt.title("Previsões vs. Valores Reais")
    plt.grid(True)
    plt.savefig(predictions_plot_path)
    plt.close()
    print(f"Gráfico Previsões vs. Reais guardado em {predictions_plot_path}")

    # Gráfico: Resíduos
    residuals = y_test - y_pred
    plt.figure(figsize=(10, 6))
    sns.histplot(residuals, kde=True)
    plt.xlabel("Resíduos (Real - Previsto)")
    plt.ylabel("Frequência")
    plt.title("Distribuição dos Resíduos")
    plt.grid(True)
    plt.savefig(residuals_plot_path)
    plt.close()
    print(f"Gráfico de Resíduos guardado em {residuals_plot_path}")

except FileNotFoundError as fnf_error:
    print(f"Erro: {fnf_error}")
except Exception as e:
    print(f"Ocorreu um erro inesperado durante a modelagem/avaliação: {e}")




A carregar dados pré-processados de /home/ubuntu/tech_challenge\02_preprocessed_data.csv...

A iniciar modelagem...
A dividir dados em conjuntos de treino e teste...
Tamanho do conjunto de treino: 1070 amostras
Tamanho do conjunto de teste: 268 amostras

A treinar o modelo de Regressão Linear...
Modelo treinado com sucesso.

A avaliar o modelo no conjunto de teste...
Mean Absolute Error (MAE): 4181.19
Mean Squared Error (MSE): 33596915.85
Root Mean Squared Error (RMSE): 5796.28
R-squared (R²): 0.7836
Métricas de avaliação guardadas em /home/ubuntu/tech_challenge\results\evaluation_metrics.txt

A realizar validação estatística com statsmodels...

Sumário do Modelo (statsmodels OLS):
                            OLS Regression Results                            
Dep. Variable:                charges   R-squared:                       0.742
Model:                            OLS   Adj. R-squared:                  0.740
Method:                 Least Squares   F-statistic:                    